In [ ]:
import os
import time
import base64
import random
import mimetypes
from io import BytesIO
from PIL import Image
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv(""),
    base_url=""
)

SYSTEM_PROMPT = [{"type": "text", "text": ""}]

INPUT_PRICE_PER_1000 = 0.0015
OUTPUT_PRICE_PER_1000 = 0.0045
MAX_IMAGE_SIZE_BYTES = 10 * 1024 * 1024
TARGET_MAX_SIZE_BYTES = 10 * 1024 * 1024

def resize_if_aspect_ratio_exceeds(image: Image.Image, max_ratio: float = 200) -> Image.Image:
    w, h = image.size
    if w / h > max_ratio:
        new_w = int(max_ratio * h)
        return image.resize((new_w, h), Image.LANCZOS)
    elif h / w > max_ratio:
        new_h = int(max_ratio * w)
        return image.resize((w, new_h), Image.LANCZOS)
    return image

def compress_image(img: Image.Image, mime_type: str, max_bytes: int = TARGET_MAX_SIZE_BYTES) -> BytesIO:
    buffer = BytesIO()
    if mime_type == "image/jpeg":
        quality = 95
        while quality >= 10:
            buffer.seek(0)
            buffer.truncate()
            img.save(buffer, format="JPEG", quality=quality)
            if buffer.tell() <= max_bytes:
                break
            quality -= 5
        buffer.seek(0)
    else:
        img.save(buffer, format=img.format or "PNG")
        buffer.seek(0)
    return buffer

def encode_image(image: Image.Image, mime_type: str) -> str:
    buffer = BytesIO()
    fmt = 'JPEG' if mime_type == 'image/jpeg' else 'PNG'
    image.save(buffer, format=fmt)
    base64_data = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return f"data:{mime_type};base64,{base64_data}"

def format_tokens(tokens):
    if tokens >= 1_000_000:
        return f"{tokens/1_000_000:.2f}百万"
    if tokens >= 1_000:
        return f"{tokens/1_000:.1f}千"
    return str(tokens)

def estimate_price(input_tokens, output_tokens):
    return round((input_tokens / 1000) * INPUT_PRICE_PER_1000 + (output_tokens / 1000) * OUTPUT_PRICE_PER_1000, 4)

def analyze_image(image_data_url, model="", max_retries=3):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": image_data_url}},
                {"type": "text", "text": ""}
            ]
        }
    ]
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(model=model, messages=messages)
            content = response.choices[0].message.content.strip()
            usage = getattr(response, "usage", None)
            in_tokens = getattr(usage, "prompt_tokens", 0) if usage else 0
            out_tokens = getattr(usage, "completion_tokens", 0) if usage else 0
            return content, in_tokens, out_tokens
        except Exception as e:
            print(f"API 调用失败（第 {attempt + 1} 次）: {e}")
            time.sleep(min(2 ** attempt, 10) * random.uniform(0.8, 1.2))
    return None, 0, 0

def process_split_images(upper_img, lower_img, mime_type):
    result_all = ""
    total_in, total_out = 0, 0

    for part, name in zip([upper_img, lower_img], ["上半部分", "下半部分"]):
        data_url = encode_image(part, mime_type)
        result, in_tok, out_tok = analyze_image(data_url)
        if result is None:
            return None, 0, 0
        result_all += f"{name}内容：{result} "
        total_in += in_tok
        total_out += out_tok

    return result_all.strip(), total_in, total_out

def process_three_split_images(img, mime_type):
    w, h = img.size
    h1 = h // 3
    h2 = 2 * h // 3
    segments = [
        img.crop((0, 0, w, h1)),
        img.crop((0, h1, w, h2)),
        img.crop((0, h2, w, h))
    ]
    parts = ["上部", "中部", "下部"]

    result_all = ""
    total_in, total_out = 0, 0

    for segment, name in zip(segments, parts):
        data_url = encode_image(segment, mime_type)
        result, in_tok, out_tok = analyze_image(data_url)
        if result is None:
            return None, 0, 0
        result_all += f"{name}内容：{result} "
        total_in += in_tok
        total_out += out_tok

    return result_all.strip(), total_in, total_out

def try_process_image(path):
    mime_type, _ = mimetypes.guess_type(path)
    if not mime_type:
        print(f"❌ 无法识别图片格式: {path}")
        return None, 0, 0

    with Image.open(path) as img:
        img = resize_if_aspect_ratio_exceeds(img)
        buffer = BytesIO()
        img.save(buffer, format='JPEG')
        size = buffer.tell()

        if size > MAX_IMAGE_SIZE_BYTES:
            buffer = compress_image(img, mime_type, TARGET_MAX_SIZE_BYTES)
            if buffer.tell() > MAX_IMAGE_SIZE_BYTES:
                w, h = img.size
                upper = img.crop((0, 0, w, h // 2))
                lower = img.crop((0, h // 2, w, h))
                return process_split_images(upper, lower, mime_type)

        data_url = encode_image(img, mime_type)
        return analyze_image(data_url)

def process_one_image(image_path, fallback_name="concat_resized.jpg"):
    result, in_tok, out_tok = try_process_image(image_path)
    if result is not None:
        return result, in_tok, out_tok

    print(f"⚠️ 主图处理失败: {image_path}，尝试备用图...")
    fallback_path = os.path.join(os.path.dirname(image_path), fallback_name)
    if not os.path.exists(fallback_path):
        print(f"❌ 备用图不存在: {fallback_path}")
        return None, 0, 0

    mime_type, _ = mimetypes.guess_type(fallback_path)
    with Image.open(fallback_path) as fallback_img:
        w, h = fallback_img.size
        if max(w / h, h / w) > 200:
            print("📐 备用图宽高比极端，使用上中下三段处理...")
            return process_three_split_images(fallback_img, mime_type)
        else:
            return try_process_image(fallback_path)

def process_folder(input_folder, output_root):
    all_images = []
    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.lower() == "concat.jpg":
                all_images.append(os.path.join(root, f))

    print(f"共发现 {len(all_images)} 张图片待处理")
    success, fail = 0, 0
    total_in, total_out, total_cost = 0, 0, 0.0

    for idx, img_path in enumerate(all_images, 1):
        print(f"\n[{idx}/{len(all_images)}] 正在处理：{img_path}")
        try:
            result, in_tok, out_tok = process_one_image(img_path)
        except Exception as e:
            result, in_tok, out_tok = None, 0, 0
            print(f"❌ 读取失败：{e}")

        rel_dir = os.path.relpath(os.path.dirname(img_path), input_folder)
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        if result is None:
            fail += 1
            failed_dir = os.path.join(output_root, rel_dir + "_处理失败")
            os.makedirs(failed_dir, exist_ok=True)
            out_path = os.path.join(failed_dir, base_name + ".txt")  
            with open(out_path, "w", encoding="utf-8") as f:
                f.write("图像处理失败或超过最大限制")
            print(f"⚠️ 已输出失败文件: {out_path}")
            continue

        out_dir = os.path.join(output_root, rel_dir)
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, base_name + ".txt")
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(result)

        success += 1
        total_in += in_tok
        total_out += out_tok
        cost = estimate_price(in_tok, out_tok)
        total_cost += cost

        print(f"✅ 成功 - 输入Token: {format_tokens(in_tok)} 输出Token: {format_tokens(out_tok)} 费用: ¥{cost:.4f}")
        print(f"结果保存在: {out_path}")

    print(f"\n🎉 处理完成：成功={success}，失败={fail}")
    print(f"💰 总Token：输入={format_tokens(total_in)} 输出={format_tokens(total_out)}")
    print(f"💸 总费用：¥{total_cost:.4f}")

# ============ 主入口 ============

if __name__ == "__main__":
    input_folder = r""
    output_root = r""
    os.makedirs(output_root, exist_ok=True)
    process_folder(input_folder, output_root)
